In [ ]:
import sys
import os
sys.path.insert(0, '../driver')

In [ ]:
import numpy as np
import math
import plotly.graph_objects as go
from control import MoveAxisRateInterpolator, ConfigurationManager

cm = ConfigurationManager()
move_axis_data = cm.calibration_data

In [ ]:
def plot_slow_interp_raw_dps(axis_id):
    axis_data = move_axis_data[axis_id]
    interpolator = MoveAxisRateInterpolator(axis_data)

    idx = axis_data["RAW"].index(5) + 1
    ascom_rates = np.linspace(axis_data["ASCOM"][0], axis_data["ASCOM"][idx - 1], 100)
    interpolated_raw = interpolator.ASCOM.SLOW(ascom_rates)
    interpolated_dps = interpolator.RAW.toDPS(ascom_rates)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=interpolated_raw,
        y=interpolated_dps,
        mode='lines',
        name='Interpolated DPS (Slow)',
        line=dict(color='royalblue'),
        hovertemplate=(
            'ASCOM Rate: %{x:.2f}<br>' +
            'RAW Rate: %{x:.2f}<br>' +
            'DPS: %{y:.6f}<extra></extra>'
        ),
    ))

    fig.add_trace(go.Scatter(
        x=axis_data["RAW"][:idx],
        y=axis_data["DPS"][:idx],
        mode='markers',
        name='Original Data',
        marker=dict(color='blue', size=6),
        hovertemplate=(
            'ASCOM Rate: %{x:.2f}<br>' +
            'RAW Rate: %{x:.2f}<br>' +
            'DPS: %{y:.6f}<extra></extra>'
        ),
    ))

    fig.update_layout(
        title=f"Axis {axis_id} — Slow Interpolation (RAW → DPS)",
        xaxis_title="RAW Rate",
        yaxis_title="Degrees Per Second (DPS)",
        template="plotly_white"
    )

    fig.show()



In [ ]:
def plot_fast_interp_raw_dps(axis_id):
    axis_data = move_axis_data[axis_id]
    interpolator = MoveAxisRateInterpolator(axis_data)

    idx = axis_data["RAW"].index(5) + 1
    raw_rates = np.linspace(6, 2500, 200)
    interpolated_dps = interpolator.RAW.toDPS(raw_rates)
    ascom_values = axis_data['ASCOM'][idx:]



    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=raw_rates,
        y=interpolated_dps,
        mode='lines',
        name='Interpolated DPS (Fast)',
        line=dict(color='royalblue'),
        hovertemplate=(
            'RAW Rate: %{x:.2f}<br>' +
            'DPS: %{y:.6f}<extra></extra>'
        ),
    ))

    fig.add_trace(go.Scatter(
        x=axis_data["RAW"][idx:],
        y=axis_data["DPS"][idx::],
        mode='markers',
        name='Original Data',
        marker=dict(color='blue', size=6),
        hovertemplate=(
            'ASCOM Rate: %{customdata[0]:.2f}<br>' +
            'RAW Rate: %{x:.2f}<br>' +
            'DPS: %{y:.6f}<extra></extra>'
        ),
        customdata=np.stack([ascom_values], axis=-1)
    ))

    fig.update_layout(
        title=f"Axis {axis_id} — Fast Interpolation (RAW → DPS)",
        xaxis_title="RAW Rate",
        yaxis_title="Degrees Per Second (DPS)",
        template="plotly_white"
    )

    fig.show()



In [ ]:
from scipy.interpolate import PchipInterpolator
def plot_fast_interp_ascom_dps(axis_id):
    axis_data = move_axis_data[axis_id]
    mid = axis_data["DPS"][5]
    max = axis_data["DPS"][-1]
    interp = PchipInterpolator([0, mid, 0.2*max, 0.5*max, max], [0,5,6,7,9], extrapolate=True)

    interpolator = MoveAxisRateInterpolator(axis_data)

    raw_rates = np.linspace(0, 2500, 200)

    interpolated_dps = interpolator.RAW.toDPS(raw_rates)
    ascom_rates = interp(interpolated_dps)
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=ascom_rates,
        y=interpolated_dps,
        mode='lines',
        name='Interpolated DPS (Fast)',
        line=dict(color='royalblue'),
        hovertemplate=(
            'ASCOM Rate: %{x:.2f}<br>' +
            'DPS: %{y:.6f}<extra></extra>'
        ),
    ))

    fig.add_trace(go.Scatter(
        x=axis_data["ASCOM"],
        y=axis_data["DPS"],
        mode='markers',
        name='Original Data',
        marker=dict(color='blue', size=6),
        hovertemplate=(
            'ASCOM Rate: %{x:.2f}<br>' +
            'DPS: %{y:.6f}<extra></extra>'
        ),
    ))

    fig.update_layout(
        title=f"Axis {axis_id} — Interpolation (ASCOM → DPS)",
        xaxis_title="ASCOM Rate",
        yaxis_title="Degrees Per Second (DPS)",
        template="plotly_white"
    )

    fig.show()



In [ ]:
axis=2
plot_slow_interp_raw_dps(axis)
plot_fast_interp_raw_dps(axis)
plot_fast_interp_ascom_dps(axis)